In [17]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request, json
import re
import matplotlib as plt
import sqlalchemy
import sqlite3
import math

## WATCH OUT, WE'RE LOADING DATA HERE

In [18]:
url_endpoint = 'https://data.cityofchicago.org/resource/cwig-ma7x.json'
response = requests.get(url_endpoint)
data = response.json()
df = pd.DataFrame(data)
df.violations = df.violations.fillna('')


In [21]:
df.columns

Index([':@computed_region_43wa_7qmu', ':@computed_region_6mkv_f3dw',
       ':@computed_region_awaf_s7ux', ':@computed_region_bdys_3d7i',
       ':@computed_region_vrxf_vc4k', 'address', 'aka_name', 'city',
       'dba_name', 'facility_type', 'inspection_date', 'inspection_id',
       'inspection_type', 'latitude', 'license_', 'location', 'longitude',
       'results', 'risk', 'state', 'violations', 'zip'],
      dtype='object')

**Create a list of infractions for each Inspection**

In [22]:
i=0
l=[]
crt = []
ser = []
mnr = []
while i < len(df):
    s = df['violations'][i]
    t1 = re.findall("\d+",str(re.findall("\|\s\d+",s)))
    ti = len([int(i) for i in t1])
    
    #Find number of infrac type
    m = len([int(i) for i in t1 if int(i) > 29 and int(i) <71])
    s = len([int(i) for i in t1 if int(i) > 14 and int(i) <30])
    c = len([int(i) for i in t1 if int(i) > 0 and int(i) <16])
    
    #Create list of vals
    l.append(ti)
    mnr.append(m)
    ser.append(s)
    crt.append(c)
    i=i+1

df['infracs']=l
df['infracs_minor'] = mnr
df['infracs_serious']= ser
df['infracs_critical']= crt

In [38]:
#TEST CELL
s = df['violations'][8]
t1 = re.findall("\d+",str(re.findall("\|\s\d+",s)))
m = len([int(i) for i in t1 if int(i) > 29 and int(i) <71])
t2

2

In [4]:
critical = range(1,15)
serious = range(15,30)
minor = range(30,70)

rl = list(testdf['results'])
testdf['results'][1]
#len([1,0 for i in rl if i == 'Pass',i != 'Pass']) #and 0 for i in r1 if i != 'Pass']

NameError: name 'testdf' is not defined

## Mission Impossible: Mr Clean

In [24]:
#Prep DataFrame for SQLite
l = range(8,len(df.columns))
testdf = df.iloc[:,l]
testdf['Pass_Fail'] = pass_fail(testdf['results'])
testdf = testdf.drop(['location','violations','results','latitude','longitude','state'], axis =1)

#Get rid of NA results and non restuarants
testdf = testdf[testdf.Pass_Fail != 'NA']
testdf = testdf[testdf.facility_type == 'Restaurant']


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [25]:
#Convert to dummies, drop unnecissary columns
risks = pd.get_dummies(testdf.risk)
itype = pd.get_dummies(testdf.inspection_type)
#NEW ZIP DUMMIES
zcode = pd.get_dummies(testdf.zip)
testdf = testdf.join(risks).join(itype).join(zcode)
testdf['itype_re'] = round((testdf['Canvass Re-Inspection']+testdf['Complaint Re-Inspection']+testdf['License Re-Inspection']+.6)/3)
testdf['itype_complaint'] = round((testdf['Short Form Complaint']+testdf['Complaint']+.1)/2)
testdf = testdf.drop(['inspection_type','facility_type','inspection_date','license_','Canvass Re-Inspection','Complaint','Complaint Re-Inspection','License Re-Inspection','Short Form Complaint','risk'], axis=1)
testdf.itype_re = testdf.itype_re.astype(int)
testdf.itype_complaint = testdf.itype_complaint.astype(int)

In [26]:
testdf

,dba_name,inspection_id,zip,infracs,infracs_minor,infracs_serious,infracs_critical,Pass_Fail,Risk 1 (High),Risk 2 (Medium),...,60655,60656,60657,60659,60660,60661,60666,60707,itype_re,itype_complaint
2,TRUENORTH CAFE,2150531,60637,7,7,0,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
5,56TH STREET HAROLD'S GRILL AND WAFFLES,2150425,60636,2,2,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
6,ALICE'S BAR B QUE,2150471,60653,5,5,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
7,SEE THRU CHINESE KITCHEN,2150488,60619,7,7,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
9,MCDONALD'S #6113,2150495,60621,5,5,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
15,ORIGINAL RAINBOW CONE INC,2150457,60620,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
19,BIG AL'S RESTAURANT,2150496,60632,2,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
21,KANELA,2150420,60640,9,7,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
25,65 EXPRESS,2150417,60606,2,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
29,HUA'S CHINESE CHOP SUEY EXPRESS,2150515,60617,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [290]:
#Test Cell
#tt = pd.merge(testdf,risks,how='inner',on=index)
nwdf = pd.DataFrame(columns=testdf.zip.unique().astype(list))

o = []
zips = testdf.zip.unique()
zipco = 60659
for z in zips:
    if zipco == int(z):
        o.append(1)
    else:
        o.append(0)

nwdf.loc[0] = o
nwdf

,60626,60613,60612,60640,60607,60617,60620,60614,60639,60659,...,60641,60637,60604,60638,60609,60606,60602,60624,60608,60707
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [23]:
#Function to bucket outcomes
def pass_fail(raw):
    output = []
    i=0
    stop = (len(raw))
    while i < stop:
        if raw[i] == 'Pass':
            output.append(1)
        elif raw[i] == 'Fail':
            output.append(0)
        else:
            output.append('NA')
        i=i+1
    return output

In [10]:
#CHECK
len(testdf['inspection_id'].unique())

487

## Database Party Zone

In [333]:
#Create DB, load Panda DF
conn = sqlite3.connect("FoodInspect.db")
c = conn.cursor()
testdf.to_sql("Inspections",conn, if_exists="replace")
t = pd.read_sql_query("select * from Inspections limit 5;", conn)
conn.close()

/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:1345: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [334]:
t

,index,dba_name,inspection_id,zip,infracs,infracs_minor,infracs_serious,infracs_critical,Pass_Fail,Risk 1 (High),...,60655,60656,60657,60659,60660,60661,60666,60707,itype_re,itype_complaint
0,0,DUNKIN DONUTS,2145400,60626,2,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,TAC QUICK,2145441,60613,5,4,1,0,1,1,...,0,0,0,0,0,0,0,0,1,0
2,2,RYHANAS CUISINE,2145440,60612,11,5,6,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,3,EL PECADO INC.,2145411,60626,7,6,0,1,1,1,...,0,0,0,0,0,0,0,0,1,0
4,4,OCTAVIO CANTINA & KITCHEN,2145430,60640,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0


In [111]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

MSIA_SETTINGS = ".\config.py"
app = Flask(__name__)
app.config.from_envvar(MSIA_SETTINGS, silent=True)
db = SQLAlchemy(app)

/anaconda/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:774: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  'Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. '
/anaconda/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [110]:
import sqlalchemy as sa
config = {'db.url':'sqlite:////Users/matthewgallagher/MSiA/Winter_2018/MSIA_423/FoodInspection/develop/FoodInspect.db', 'db.echo':'True'}
engine = sa.engine_from_config(config, prefix='db.')
testdf.to_sql("Inspections",engine,if_exists="replace")

2018-02-20 14:59:32,504 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-02-20 14:59:32,506 INFO sqlalchemy.engine.base.Engine ()
2018-02-20 14:59:32,510 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-02-20 14:59:32,511 INFO sqlalchemy.engine.base.Engine ()
2018-02-20 14:59:32,514 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Inspections")
2018-02-20 14:59:32,515 INFO sqlalchemy.engine.base.Engine ()
2018-02-20 14:59:32,520 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Inspections")
2018-02-20 14:59:32,521 INFO sqlalchemy.engine.base.Engine ()
2018-02-20 14:59:32,524 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-02-20 14:59:32,525 INFO sqlalchemy.engine.base.Engine ()
2018-02-20 14:59:32,528 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Inspections")
2018-02-20 14:59:32,530 INFO sqlalchemy.engine.base.E

2018-02-20 14:59:32,795 INFO sqlalchemy.engine.base.Engine COMMIT
2018-02-20 14:59:32,800 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-02-20 14:59:32,801 INFO sqlalchemy.engine.base.Engine ()


## First stab at a model

In [27]:
from sklearn import linear_model
from sklearn import model_selection
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import cross_val_score

In [28]:
#Create Model
Y = testdf['Pass_Fail']
Y=Y.astype('int')
X = testdf.drop(['inspection_id','dba_name','Pass_Fail','inspection_id','zip'],axis=1)
clf = linear_model.LogisticRegression()
m_fit = clf.fit(X, Y)
y_pred = m_fit.predict(X)

In [29]:
#Priors
p = len(testdf[testdf.Pass_Fail == 1])
f = len(testdf[testdf.Pass_Fail == 0])
priors = [p/(p+f) , f/(p+f)]
priors

[0.7078891257995735, 0.2921108742004264]

In [30]:
#Prediction Outcomes
print('Accuracy of logistic regression classifier on training set: {:.2f}'.format(m_fit.score(X,Y)))

from sklearn.metrics import *
confusion_matrix = confusion_matrix(Y, y_pred)
print(confusion_matrix)

Accuracy of logistic regression classifier on training set: 0.87
[[ 83  54]
 [  7 325]]


In [16]:
#CV outcome
kfold = model_selection.KFold(n_splits=10, random_state=7)
scoring = 'accuracy'
results = model_selection.cross_val_score(clf,X,Y, cv=kfold, scoring=scoring)
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

10-fold cross validation average accuracy: 0.854


## Python pickles

In [320]:
import pickle

#Pickle class test cell
class model_reqs:
    def __init__(self, model , specs):
        self.model = model
        self.specs = list(specs)

specification = model_reqs(m_fit,testdf.columns)
#specification.specs[14:len(specification.specs)-2]

pickling_on = open("Model.pickle","wb")
pickle.dump(specification, pickling_on)
pickling_on.close()

In [321]:
model_ = pickle.load(open('/Users/matthewgallagher/MSiA/Winter_2018/MSIA_423/FoodInspection/develop/notebooks/Model.pickle','rb'))
model = model_.model
specs = model_.specs